<a href="https://colab.research.google.com/github/Juuliee/ABSA-PyTorch/blob/master/entity_aspect_pair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Import Packages

In [5]:
import pandas as pd
import numpy as np

## Import Data

without unnecessary rows and columns.

In [6]:
original_data = pd.read_excel('/content/drive/My Drive/Review Classifier - Trainingsset-CCG-3.xlsx')

In [7]:
print(original_data[original_data.duplicated(keep= False)]) # row 194 and 202 duplicate

       Review Aspect Aspect 1  ... Aspect 3 Sentiment Aspect 3 Sub-category Aspect 3
194  Gute app    NaN      NaN  ...      NaN                NaN                   NaN
202  Gute app    NaN      NaN  ...      NaN                NaN                   NaN

[2 rows x 11 columns]


In [8]:
def drop_rows(my_data):

    lorem_indices = my_data[my_data["Feedback"] == ' Lorem ipsum dolor sit amet\\\\  consectetur adipiscing elit. '].index
    new_data = my_data.drop(lorem_indices)
    new_data.drop_duplicates(inplace=True,ignore_index=True)
    new_data.reset_index(drop=True, inplace=True)

    return new_data

In [34]:
entity_data = original_data.rename(columns={"Review": "Feedback", "Aspect 1": "Aspect_1", "Sentiment Aspect 1": "Sentiment_1", "Aspect 2": "Aspect_2", "Sentiment Aspect 2": "Sentiment_2", "Aspect 3": "Aspect_3", "Sentiment Aspect 3": "Sentiment_3", "Sub-category Aspect 1": "Entity_1", "Sub-category Aspect 2": "Entity_2", "Sub-category Aspect 3": "Entity_3"})
entity_data = drop_rows(entity_data)

## Remove special characters

In [35]:
def remove_special_characters(my_data):

    my_data["Feedback"] = my_data["Feedback"].str.replace('_',' ').str.replace('\\\\','')
    my_data["Feedback"] = my_data["Feedback"].str.replace('<br>',' ')
    my_data["Feedback"] = my_data["Feedback"].str.lstrip().str.rstrip() # remove leading and trailing whitespaces

    return my_data

In [ ]:
entity_data = remove_special_characters(entity_data)

In [169]:
entity_data[entity_data.Aspect_1 == 'Mitarbeiter']

,Feedback,Aspect,Aspect_1,Sentiment_1,Entity_1,Aspect_2,Sentiment_2,Entity_2,Aspect_3,Sentiment_3,Entity_3
24,Morgens den Wagen gebracht kurze Absprache mit...,"Mitarbeiter,klasse",Mitarbeiter,positive,Freundlichkeit,NaN,NaN,NaN,NaN,NaN,NaN
346,Nach Meldung des Schadens bei der Würtembergis...,NaN,Mitarbeiter,positive,Freundlichkeit,Ersatzfahrzeug,positive,Leistung,Reparatur,positive,Leistung
432,Total unfreundlich mit fast schon einer Arroga...,NaN,Mitarbeiter,negative,Freundlichkeit,Aussage,negative,Beratung,NaN,NaN,NaN
605,Keine kompetenten Mitarbeiter. Angeforderten I...,"Mitarbeiter,Informationen",Mitarbeiter,negative,Kompetenz,Information,negative,Kompetenz,NaN,NaN,NaN
616,Die Erfahrung daß Sie unter Pensionären n...,Mitarbeiter,Mitarbeiter,neutral,Qualität,Abschluss,negative,Leistung,NaN,NaN,NaN


# Count aspect, entities and entity-aspect pairs

In [171]:
# check where Entity_1 is NaN, but Entity_2, or Entity_3 is not NaN
temp = entity_data[entity_data.Entity_1.isnull()]
[temp[temp.Entity_2.isnull()]['Entity_2'].value_counts(),temp[temp.Entity_3.isnull()]['Entity_3'].value_counts()]

[Series([], Name: Entity_2, dtype: int64),
 Series([], Name: Entity_3, dtype: int64)]

In [ ]:
#Use only entity-relevant rows
entity_train =entity_data[~entity_data.Entity_1.isnull()]
entity_train.reset_index(drop=True, inplace=True)
len(entity_train)

##Add Entity Column

add column Entity showing existing entities in each Review accordingly
- due to mistakes in column Aspect, add column Aspect_new gathering aspects in each Review

In [170]:
def add_entity_column(my_data):
    
    none_list = [None] * len(my_data)
    my_data["Entity"] = none_list
    my_data["Aspect_new"] = none_list

    for ii in range(len(my_data)):
        if type(my_data["Entity_3"][ii]) == str:
           if type(my_data["Entity_2"][ii]) == str:
              my_data["Entity"][ii] = (my_data["Entity_3"][ii]+','+my_data["Entity_2"][ii]+','+my_data["Entity_1"][ii])
              my_data["Aspect_new"][ii] = (my_data["Aspect_3"][ii]+','+my_data["Aspect_2"][ii]+','+my_data["Aspect_1"][ii])
              
        elif type(my_data["Entity_2"][ii]) == str:
            my_data["Entity"][ii] = (my_data["Entity_2"][ii]+','+my_data["Entity_1"][ii])
            my_data["Aspect_new"][ii] = (my_data["Aspect_2"][ii]+','+my_data["Aspect_1"][ii])
              
        elif type(my_data["Entity_1"][ii]) == str:
            my_data["Entity"][ii] = (my_data["Entity_1"][ii])
            my_data["Aspect_new"][ii] = (my_data["Aspect_1"][ii])
        else:
            my_data = my_data.drop(ii)

    my_data.reset_index(inplace=True, drop=True)

    return my_data

In [183]:
entity_train = add_entity_column(entity_train)

,Feedback,Aspect,Aspect_1,Sentiment_1,Entity_1,Aspect_2,Sentiment_2,Entity_2,Aspect_3,Sentiment_3,Entity_3,Entity,Aspect_new
0,Ich war bei einem großen deutschen Versicherun...,"Beiträge,Geld",Beitrag,neutral,Preis,Geld,positive,Preis,NaN,NaN,NaN,"Preis,Preis","Geld,Beitrag"
1,Lt. HUK-Schreiben benötige ich die Super-PIN u...,"Bereich,Problem",Bereich,neutral,Beratung,Problem,negative,Problemlösung,NaN,NaN,NaN,"Problemlösung,Beratung","Problem,Bereich"
2,Bisher eine unkomplizierte Abwicklung - auch i...,"Abwicklung,Schadensfall",Abwicklung,positive,Leistung,Schadensfall,positive,Leistung,NaN,NaN,NaN,"Leistung,Leistung","Schadensfall,Abwicklung"
3,Schnelle Beratung und Erstellung der eVB samt...,"Beratung,Unterlagen",Beratung,positive,Beratung,Unterlagen,positive,Erreichbarkeit,NaN,NaN,NaN,"Erreichbarkeit,Beratung","Unterlagen,Beratung"
4,Der Service der RechnungsApp ist super. Die Be...,"Service,Bearbeitung",Service,positive,Erreichbarkeit,Bearbeitung,positive,Erreichbarkeit,NaN,NaN,NaN,"Erreichbarkeit,Erreichbarkeit","Bearbeitung,Service"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,Die telefonische Erreichbarkeit könnte verbess...,Erreichbarkeit,Erreichbarkeit,negative,Erreichbarkeit,Freundlichkeit,negative,Freundlichkeit,NaN,NaN,NaN,"Freundlichkeit,Erreichbarkeit","Freundlichkeit,Erreichbarkeit"
579,Die Preissteigerung nach einem Unfall erstaunt...,"Unfall,Versicherung,Formular",Unfall,negative,Preis,Versicherung,negative,Problemlösung,Formular,negative,Problemlösung,"Problemlösung,Problemlösung,Preis","Formular,Versicherung,Unfall"
580,Sehr hohe Summe die ich nach zahlen musste...,NaN,Summe,negative,Preis,NaN,NaN,NaN,NaN,NaN,NaN,Preis,Summe
581,Die grüne Versichertenkarte wurde fehlerhaft a...,NaN,Karte,negative,Qualität,Preis,positive,Preis,NaN,NaN,NaN,"Preis,Qualität","Preis,Karte"


In [243]:
def entity_aspect_tuple(my_data):

    none_list = [None] * len(my_data)
    my_data["Entity_Aspect_Tuple"] = none_list

    for ii in range(len(my_data)):
        word = my_data["Entity"][ii].lower().split(sep=',')
        temp = ()
        for x in range(len(word)):
            temp += (my_data["Aspect_new"][ii].lower().split(sep=',')[x].rstrip(),word[x])
        my_data["Entity_Aspect_Tuple"][ii] = temp

    return my_data

In [244]:
my_data["Aspect_new"][ii].lower().split(sep=',')[1].rstrip()
my_data["Aspect_new"][580].lower().split(sep=',')
#my_data["Entity_Aspect_Tuple"][ii] = temp

['summe']

In [245]:
entity_aspect_tuple(entity_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


AttributeError: ignored

## Count aspect and entity frequency

In [175]:
# frequencycount occurence
def occurDict(items):
    d = {}
    for i in items:
        if i in d:
            d[i] = d[i]+1
        else:
            d[i] = 1
    return d

In [179]:
def aspect_entity_frequency(my_data):

    sep_aspects = []
    aspect_freq_list = []
    sep_entities =[]
    entity_freq_list = []
    #aspect_entity_index_dict =[]


    for ii in range(len(my_data)):

        if type(my_data.Entity[ii]) == str:
            sep_entities += my_data["Entity"][ii].lower().split(sep=',')
# this section have to be considered again later.
            if type(my_data.Aspect[ii]) == str:
               sep_aspects += my_data["Aspect_new"][ii].lower().split(sep=',')
            #else:
               # add aspect "no_aspect" to reviews without any aspect
               #tuple_list += [tuple(["no_entity", ])]

    aspect_freq_dict = occurDict(sep_aspects)
    aspect_freq_list = sorted((value,key) for (key,value) in aspect_freq_dict.items())

    entity_freq_dict = occurDict(sep_entities)
    entity_freq_list = sorted((value,key) for (key,value) in entity_freq_dict.items())
          
    return sep_aspects,sep_entities,aspect_freq_list,entity_freq_list

In [180]:
sep_aspects, sep_entities, aspect_freq_list, entity_freq_list = aspect_entity_frequency(entity_train)
[len(set(sep_aspects)), len(sep_aspects)]

[245, 911]

In [181]:
entity_freq_list

[(30, 'freundlichkeit'),
 (41, 'kompetenz'),
 (107, 'beratung'),
 (138, 'preis'),
 (146, 'problemlösung'),
 (188, 'erreichbarkeit'),
 (367, 'qualität'),
 (371, 'leistung')]

In [182]:
aspect_freq_list

[(1, 'abgewickelt'),
 (1, 'ablehnung'),
 (1, 'abzocke'),
 (1, 'abzüge'),
 (1, 'agentur'),
 (1, 'ahnung'),
 (1, 'akku'),
 (1, 'anliegen'),
 (1, 'anmeldung'),
 (1, 'anruf'),
 (1, 'anspruch'),
 (1, 'antwort '),
 (1, 'argumente'),
 (1, 'aufbau'),
 (1, 'aufwand'),
 (1, 'auswahl'),
 (1, 'auto'),
 (1, 'bankverbindung'),
 (1, 'beantwortung'),
 (1, 'bedienung'),
 (1, 'beitrag '),
 (1, 'benachrichtigung'),
 (1, 'benzinsparen'),
 (1, 'berater'),
 (1, 'berechnung'),
 (1, 'beschreibung'),
 (1, 'betrag'),
 (1, 'bezahlung'),
 (1, 'bluetoothverbindung'),
 (1, 'brief'),
 (1, 'dokumente'),
 (1, 'eindruck'),
 (1, 'einstufung'),
 (1, 'einwandfrei'),
 (1, 'empfehlung'),
 (1, 'erfolg'),
 (1, 'erwartung'),
 (1, 'fahrt'),
 (1, 'fahrzeug'),
 (1, 'feedback '),
 (1, 'formular'),
 (1, 'frage'),
 (1, 'freundlichkeit'),
 (1, 'funktionert'),
 (1, 'funktionioniert'),
 (1, 'führung'),
 (1, 'haftpflichtschaden'),
 (1, 'hilfestellung'),
 (1, 'informationen'),
 (1, 'internetportal'),
 (1, 'jahresbeitrag'),
 (1, 'jahresbe

##creat entity-aspect pairs

entity-aspect assigner

In [ ]:
def entity_aspect_assigner(my_data):

   return 

In [127]:
nones = np.zeros(shape=(0,10))
data = pd.DataFrame(nones, columns=["Aspect", "Aspect_Freq", 
                                    "Beratung", "Erreichbarkeit", "Freundlichkeit", "Kompetenz", 
                                    "Qualität", "Preis","Leistung/Nutzung", "Anforderung"])

In [158]:
data =pd.DataFrame(aspect_freq_list,columns=['Aspect_Freq','Aspect'])
data=data.assign(Beratung ="", Erreichbarkeit="", Freundlichkeit="", Kompetenz="", Qualität="", Preis="",Problemlösung="", Anforderung="")

,Aspect_Freq,Aspect,Beratung,Erreichbarkeit,Freundlichkeit,Kompetenz,Qualität,Preis,Problemlösung,Anforderung
0,1,mitarbeiter,,,,,,,,
1,1,abzocke,,,,,,,,
2,1,abzüge,,,,,,,,
3,1,alte,,,,,,,,
4,1,angegeben,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
191,25,funktioniert,,,,,,,,
192,27,abwicklung,,,,,,,,
193,33,kfz,,,,,,,,
194,60,versicherung,,,,,,,,


In [183]:
for ii in range(len(entity_train)):
  if entity_train.

583

#Create aspect list and tuples

In [ ]:
def aspect_tuple_generator(my_data):

    sep_aspects = []
    tuple_list = []

    for ii in range(len(my_data)):

        if type(my_data.Aspect[ii]) == str:
            sep_aspects += my_data["Aspect"][ii].lower().split(sep=',')
            # split list of aspects and make a tuple out of them
            tuple_list += [tuple(my_data.Aspect[ii].lower().split(sep=','))]
        else:
            # add aspect "no_aspect" to reviews without any aspect
            tuple_list += [tuple(["no_aspect", ])]
          
    return sep_aspects, tuple_list

In [ ]:
sep_aspects, aspects_in_tuples = aspect_tuple_generator(kfz_data)

##Get List of rare Aspects

In [138]:
def aspect_list_cleaner(min_appearance_freq):

    # create a frequency list for aspects and separate into two lists: 
    # cleaned_aspect_list & rare_aspect_list

    aspect_freq_list = [[x,sep_aspects.count(x)] for x in set(sep_aspects)]

    cleaned_aspect_list = [] 
    rare_aspect_list = []

    for ii in range(len(aspect_freq_list)):

        # each aspect has to appear at least min_appearance_freq time within
        # the data to stay in the cleaned_aspect_list
        # if min_appearance_freq = 1, no aspects are removed
        if aspect_freq_list[ii][1] < min_appearance_freq: 
            rare_aspect_list += aspect_freq_list[ii][0:-1]
        else:
            cleaned_aspect_list += aspect_freq_list[ii][0:-1]
    
    return aspect_freq_list, cleaned_aspect_list, rare_aspect_list

In [ ]:
#Choose how often an aspect has to appear at least.
MIN_APP_FREQ = 1
aspect_freq_list, cleaned_aspect_list, rare_aspect_list = aspect_list_cleaner(MIN_APP_FREQ)
# Check length of separated lists
[len(cleaned_aspect_list),len(rare_aspect_list)]

##Remove rare aspects from tuple List

In [142]:
def rare_aspect_remover(tuple_list, rare_aspect_list):

    # create a new tuple_list: cleaned_aspects, removing rare aspects that just appear once
    cleaned_aspects = [None] * len(tuple_list)

    for ii in range(len(tuple_list)):
        cleaned_aspects[ii] = tuple()
        for ind in tuple_list[ii]:
            if ind not in rare_aspect_list:
                cleaned_aspects[ii] += tuple([ind])
        # if the removed aspect was the only aspect, we need to label it "no_aspect"
        if cleaned_aspects[ii] == ():
            cleaned_aspects[ii] = tuple(["no_aspect", ])
    
    return cleaned_aspects

In [ ]:
cleaned_aspects = rare_aspect_remover(aspects_in_tuples, rare_aspect_list)

Add columns to data set

In [ ]:
def aspect_encoder(my_data, cleaned_aspects):

    none_list = [None] * len(cleaned_aspects)
    tuple_df = pd.DataFrame(none_list, columns=["Tuples"])
    tuple_df.Tuples = cleaned_aspects

    my_data["Cleaned_Aspects"] = tuple_df.Tuples

    mlb = MultiLabelBinarizer()
    encoded_aspects = mlb.fit_transform(my_data["Cleaned_Aspects"])
    aspect_table = pd.DataFrame(encoded_aspects, columns=mlb.classes_)

    my_data["Aspect_Encoding"] = aspect_table.values.tolist()

    return my_data, aspect_table, mlb.classes_

In [ ]:
kfz_data, aspect_table, aspects = aspect_encoder(kfz_data, cleaned_aspects)